<a href="https://colab.research.google.com/github/JavierDiezGonzalez/Algoritmos-de-Optimizacion/blob/master/AG2_Javier%20D%C3%ADez%20Gonz%C3%A1lez.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Nombre: Javier Díez González
## URL: https://colab.research.google.com/drive/1Sc9GgAAlUzQrJdhJ851g1cIIbRuyiS6r
## Github:https://github.com/JavierDiezGonzalez/Algoritmos-de-Optimizacion

#Viaje por el río. Programación dinámica


In [0]:
TARIFAS = [
[0,5,4,3,999,999,999],
[999,0,999,2,3,999,11],
[999,999, 0,1,999,4,10],
[999,999,999, 0,5,6,9],
[999,999, 999,999,0,999,4],
[999,999, 999,999,999,0,3],
[999,999,999,999,999,999,0]
]

In [0]:
def Precios(TARIFAS):
  
  N=len(TARIFAS[0])
  
  
  PRECIOS=[[9999]*N for i in [9999]*N]
  RUTA=[['']*N for i in ['']*N]
  
  
  for i in range(N-1):
    for j in range(i+1,N):
      MIN = TARIFAS[i][j]
      RUTA [i][j]= i
      
      for k in range(i,j):
        if PRECIOS[i][k]+TARIFAS[k][j]<MIN:
          MIN=PRECIOS[i][k]+TARIFAS[k][j]
          RUTA [i][j]=k
      PRECIOS[i][j]=MIN
  return PRECIOS,RUTA

PRECIOS,RUTA=Precios(TARIFAS)

def calcular_ruta(RUTA,desde,hasta):
  if desde==hasta:
    return desde
  else:
    return str(calcular_ruta(RUTA,desde, RUTA[desde][hasta]))+''+str( RUTA[desde][hasta])

calcular_ruta(RUTA,0,6)

'0025'

In [0]:
print(PRECIOS)
print(RUTA)

[[9999, 5, 4, 3, 8, 8, 11], [9999, 9999, 999, 2, 3, 8, 7], [9999, 9999, 9999, 1, 6, 4, 7], [9999, 9999, 9999, 9999, 5, 6, 9], [9999, 9999, 9999, 9999, 9999, 999, 4], [9999, 9999, 9999, 9999, 9999, 9999, 3], [9999, 9999, 9999, 9999, 9999, 9999, 9999]]
[['', 0, 0, 0, 1, 2, 5], ['', '', 1, 1, 1, 3, 4], ['', '', '', 2, 3, 2, 5], ['', '', '', '', 3, 3, 3], ['', '', '', '', '', 4, 4], ['', '', '', '', '', '', 5], ['', '', '', '', '', '', '']]


#Asignación de tareas. Ramificación y poda.

In [0]:
import itertools
import numpy as np

def calcular_tiempo(f):
    @wraps(f)
    def cronometro(*args, **kwargs):
        t_inicial = time()
        salida = f(*args, **kwargs)
        t_final = time()
        print('Tiempo transcurrido (en segundos): {}'.format(t_final - t_inicial))
        return salida
    return cronometro

In [0]:
COSTES=[[11,12,18,40],
        [14,15,13,22],
        [11,17,19,23],
        [17,14,20,28]]

In [0]:
def valor(S,COSTES):
  VALOR=0
  for i in range(len(S)):
    VALOR+=COSTES[S[i]][i]
  return VALOR
    
  


In [0]:

def fuerza_bruta(COSTES):
  
  mejor_valor= 10e10
  mejor_solucion=''
  
  for S in list(itertools.permutations(range(len(COSTES)))):
    valor_temp=valor(S,COSTES)
    if valor_temp<mejor_valor:
      mejor_solucion=S
      mejor_valor=valor_temp
  print('La mejor solución es :', mejor_solucion, 'con valor: ', mejor_valor)
  


  
  

In [0]:
fuerza_bruta(COSTES)

La mejor solución es : (0, 3, 1, 2) con valor:  61


Tiene un orden de N!

In [0]:
def CI(S,COSTES):
  
  VALOR=0
  
  for i in range(len(S)):
    VALOR+=COSTES[i][S[i]]
    
  for i in range(len(COSTES)):
    if i not in S:
      VALOR+=min([COSTES[j][i] for j in range(len(S),len(COSTES))])
  
  return VALOR



def crear_hijos(NODO, N):
  HIJOS=[]
  
  for i in range(N):
    if i not in NODO:
      HIJOS.append({'s':NODO+(i,)})
  
  return HIJOS

crear_hijos((0,2),4)

[{'s': (0, 2, 1)}, {'s': (0, 2, 3)}]

In [0]:
def ramificacion_y_poda(COSTES):
  DIMENSION=len(COSTES)
  #(0,1,2,3)
  MEJOR_SOLUCION=tuple(i for i in range (DIMENSION))
  
  CotaSup=valor(MEJOR_SOLUCION,COSTES)
  
  NODOS=[]
  NODOS.append({'s':(),'CI': CI((),COSTES)})
  
  iteracion=0
  
  while (len(NODOS)>0):
    iteracion+=1
    #print('\n#',iteracion)
    nodo_prometedor=min(NODOS,key=lambda x: x['CI'])
    #print(nodo_prometedor)
    #Hijos
    HIJOS=[{'s':x['s'], 'CI':CI(x['s'],COSTES)} for x in crear_hijos(nodo_prometedor['s'], DIMENSION)]
    
    NODO_FINAL=[x for x in HIJOS if len(x['s'])==DIMENSION]
    if len(NODO_FINAL)>0:
      if NODO_FINAL[0]['CI']<CotaSup:
        CotaSup=NODO_FINAL[0]['CI']
        MEJOR_SOLUCION=NODO_FINAL[0]
        
    #Poda
    HIJOS=[x for x in HIJOS if x['CI']<CotaSup]
    
    #Eliminamos el nodo expandido
    NODOS=[x for x in NODOS if x['s']!=nodo_prometedor['s']]
    
    #Incrementamos la lista de nodos pendientes de analizar
    NODOS.extend(HIJOS)
    #Se oculta este print de la práctica para que no se pierda tiempo ejecutando el algoritmo y poder comparar en realidad fuerza_bruta y ramificacion_y_poda
    #print(NODOS)
  
  print('La mejor solución es: ', MEJOR_SOLUCION)
  
ramificacion_y_poda(COSTES)

La mejor solución es:  {'s': (0, 2, 3, 1), 'CI': 61}


GENERAR MATRICES DE MAYORES DIMENSIONES Y EJECUTAR AMBOS ALGORITMOS

In [0]:
def nueva_dimension(DIMENSION):
  for i in range(DIMENSION):
    COSTES=np.random.randint(30,size=(DIMENSION,DIMENSION))
  return COSTES

In [0]:
COSTES=nueva_dimension(6)

In [0]:
fuerza_bruta(COSTES)

La mejor solución es : (0, 3, 5, 4, 1, 2) con valor:  45


In [0]:
ramificacion_y_poda(COSTES)

La mejor solución es:  {'s': (0, 4, 5, 1, 3, 2), 'CI': 45}


In [0]:
COSTES=nueva_dimension(8)

In [0]:
fuerza_bruta(COSTES)

La mejor solución es : (3, 2, 0, 5, 7, 4, 6, 1) con valor:  50


In [0]:
ramificacion_y_poda(COSTES)

La mejor solución es:  {'s': (2, 7, 1, 0, 5, 3, 6, 4), 'CI': 50}


In [0]:
COSTES=nueva_dimension(20)